In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [24]:
links = pd.read_csv('../data/links.csv', names = ['link'], skiprows=1)

In [25]:
links

,link
0,https://onebite.app/restaurant/pizza-hut-adana...
1,https://onebite.app/restaurant/del-sole-01-472...
2,https://onebite.app/restaurant/lava-lounge-piz...
3,https://onebite.app/restaurant/pizzaria-cosa-n...
4,https://onebite.app/restaurant/olivia-tjuvholm...
...,...
50582,https://onebite.app/restaurant/very-italian-pi...
50583,https://onebite.app/restaurant/donatellos-leid...
50584,https://onebite.app/restaurant/santa-lucia-lim...
50585,https://onebite.app/restaurant/pizze-pronto-sc...


In [211]:
start = time.time()

dave_scores = []
user_scores = []
name = []
address = []

for link in list(links['link']):

    r = requests.get(link)
    bs = BeautifulSoup(r.text)

    temp = []

    for elem in bs.find_all('a'):
        if 'jsx-84601126' in (elem.attrs['class']):
            temp.append(elem.text)

    name.append(temp[0])
    address.append(temp[1])

    temp2 = []

    for elem in bs.find_all('span'):
        if 'score' in (elem.attrs['class']):
            temp2.append(elem.text)

    dave_scores.append(temp2[0])
    user_scores.append(temp2[1])
    
    time.sleep(0.5)
    
end = time.time()
    
print(end-start)

23.245351791381836


In [212]:
df = pd.DataFrame({'restaurant_name': name,
                   'dave_scores': dave_scores,
                   'user_scores': user_scores,
                   'address': address})

df

,restaurant_name,dave_scores,user_scores,address
0,Pizza Hut,0.0,2.7,"213. Sok., Adana, 01"
1,Del sole,0.0,7.3,"字ニセコ483-1, 虻田郡, 01"
2,Lava Lounge Pizza Niseko,0.0,4.2,"倶知安町字山田191-19, 虻田郡, 01"
3,Pizzaria Cosa Nostra,0.0,8.5,"R. Custódio Brás Pacheco, 1, Vila Nova de Milf..."
4,Olivia Tjuvholmen,0.0,8.4,"Bryggegangen 4, Oslo, 03"
5,Mamma Pizza Osteria,0.0,9.7,"Dronningens gate 22, Oslo, 03"
6,Villa Paradiso Frogner,0.0,9.5,"Sommerrogata 17, Oslo, 03"
7,Balkan Pizza & Kebabhouse,0.0,9.3,"Mauritz Hansens gate 5, Oslo, 03"
8,Pizza Pili,0.0,9.2,"24 Rue Benoit Bunico, Nice, 06"
9,La Pizza Cresci,0.0,6.2,"34 Rue Masséna, Nice, 06"
